<a href="https://colab.research.google.com/github/asmit-ayank/UML501/blob/main/Assignment_3_UML501.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3 - Asmit Ayank (102497002)

In [ ]:
import pandas as pd
data = pd.read_csv('/content/USA_Housing.csv')
data

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price
0,79545.45857,5.682861,7.009188,4.09,23086.80050,1.059034e+06
1,79248.64245,6.002900,6.730821,3.09,40173.07217,1.505891e+06
2,61287.06718,5.865890,8.512727,5.13,36882.15940,1.058988e+06
3,63345.24005,7.188236,5.586729,3.26,34310.24283,1.260617e+06
4,59982.19723,5.040555,7.839388,4.23,26354.10947,6.309435e+05
...,...,...,...,...,...,...
4995,60567.94414,7.830362,6.137356,3.46,22837.36103,1.060194e+06
4996,78491.27543,6.999135,6.576763,4.02,25616.11549,1.482618e+06
4997,63390.68689,7.250591,4.805081,2.13,33266.14549,1.030730e+06
4998,68001.33124,5.534388,7.130144,5.44,42625.62016,1.198657e+06


In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [ ]:
X = data.drop('Price', axis=1)
y = data['Price']

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
X_scaled = np.hstack((np.ones((X_scaled.shape[0], 1)), X_scaled))

In [ ]:
def create_folds(X_data, y_data, k=5):
    n = len(X_data)
    fold_size = n // k
    folds = []
    for i in range(k):
        start, end = i * fold_size, (i + 1) * fold_size
        X_test, y_test = X_data[start:end], y_data[start:end]
        if start == 0:   # first fold
            X_train, y_train = X_data[end:], y_data[end:]
        elif end == n:   # last fold
            X_train, y_train = X_data[:start], y_data[:start]
        else:            # middle folds
            X_train = np.vstack((X_data[:start], X_data[end:]))
            y_train = np.vstack((y_data[:start].values.reshape(-1, 1), y_data[end:].values.reshape(-1, 1)))

        folds.append((X_train, y_train, X_test, y_test))

    return folds

In [ ]:
# Compute beta using Normal Equation
def compute_beta(X, y):
    XtX = np.dot(X.T, X)
    Xty = np.dot(X.T, y)
    beta = np.linalg.inv(XtX).dot(Xty)
    return beta

# Prediction
def predict(X, beta):
    print(X.shape)
    print(beta.shape)
    return np.dot(X, beta)

# R2 Score
def r2_score(y_true, y_pred):
    y_true = y_true.flatten() if hasattr(y_true, 'flatten') else np.asarray(y_true).flatten()
    y_pred = y_pred.flatten() if hasattr(y_pred, 'flatten') else np.asarray(y_pred).flatten()

    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - (ss_res / ss_tot)

# Perform 5-fold CV
folds = create_folds(X_scaled, y)
r2_scores = []
betas = []

for i, (X_train, y_train, X_test, y_test) in enumerate(folds):
    beta = compute_beta(X_train, y_train)
    y_pred = predict(X_test, beta)
    r2 = r2_score(y_test, y_pred)

    r2_scores.append(r2)
    betas.append(beta)

    print(f"Fold {i+1}:")
    print(beta.flatten()[:5])
    print("R2 Score:", r2, "\n")

# Best fold
best_index = np.argmax(r2_scores)
print("Best Fold:", best_index+1, "R2 =", r2_scores[best_index])
best_beta = betas[best_index]

(1000, 6)
(6,)
Fold 1:
[1231872.07704612  229923.70318635  164052.98579011  120923.57426579
    3533.77470973]
R2 Score: 0.9175899480765107 

(1000, 6)
(6, 1)
Fold 2:
[1232586.40453402  228365.32287054  165201.14881927  122539.46855211
    1697.03433458]
R2 Score: 0.9203015496401129 

(1000, 6)
(6, 1)
Fold 3:
[1231448.09063371  230730.20810582  163554.55706976  121669.08713233
    2620.25215054]
R2 Score: 0.9152429915320014 

(1000, 6)
(6, 1)
Fold 4:
[1.23144707e+06 2.29921558e+05 1.64523054e+05 1.19737507e+05
 1.12425659e+03]
R2 Score: 0.9208503836977656 

(1000, 6)
(6,)
Fold 5:
[1233028.38536436  230778.59921985  163755.13607252  121885.1351222
    1247.01304382]
R2 Score: 0.9138111758717495 

Best Fold: 4 R2 = 0.9208503836977656


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
final_beta = compute_beta(X_train, y_train)
y_pred_test = predict(X_test, final_beta)
final_r2 = r2_score(y_test, y_pred_test)


(1500, 6)
(6,)


In [ ]:
final_r2

np.float64(0.9146818498916266)

Question 2

In [ ]:
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.30, random_state=123, shuffle=True)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=(14/70), random_state=123, shuffle=True)

In [ ]:
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_val_s   = scaler.transform(X_val)
X_test_s  = scaler.transform(X_test)

In [ ]:
def add_intercept(X):
    return np.hstack([np.ones((X.shape[0], 1)), X])

In [ ]:
X_train_s = add_intercept(X_train_s)
X_val_s   = add_intercept(X_val_s)
X_test_s  = add_intercept(X_test_s)

In [ ]:
X = data.drop(columns=['Price']).to_numpy()
y = data['Price'].to_numpy().reshape(-1, 1)

In [ ]:
def compute_gradient(X, y, beta):
    m = X.shape[0]
    preds = X.dot(beta)           # (m,1)
    error = preds - y             # (m,1)
    grad = (2.0 / m) * (X.T.dot(error))  # (d,1)
    return grad

learning_rates = [0.001, 0.01, 0.1, 1]
num_iters = 1000
results = []

d = X_train_s.shape[1]

for lr in learning_rates:
    beta = np.zeros((d, 1), dtype=float)
    for it in range(num_iters):
        grad = compute_gradient(X_train_s, y_train, beta)
        beta = beta - lr * grad

    # compute predictions and R2
    y_val_pred  = X_val_s.dot(beta)
    y_test_pred = X_test_s.dot(beta)
    r2_val = r2_score(y_val, y_val_pred)
    r2_test = r2_score(y_test, y_test_pred)

    results.append({'lr': lr, 'beta': beta.copy(), 'r2_val': r2_val, 'r2_test': r2_test})


best_idx = int(np.argmax([r['r2_val'] for r in results]))
best = results[best_idx]

/tmp/ipython-input-2178181123.py:19: RuntimeWarning: overflow encountered in square
  ss_res = np.sum((y_true - y_pred) ** 2)


In [ ]:
best

{'lr': 0.01,
 'beta': array([[1234660.28925392],
        [ 230664.19221252],
        [ 162741.32354906],
        [ 119289.78773129],
        [   3456.43908272],
        [ 152502.41192948]]),
 'r2_val': np.float64(0.9228842709756091),
 'r2_test': np.float64(0.9194366786375839)}

Question 3

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score

In [ ]:
columns = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration",
           "num_doors", "body_style", "drive_wheels", "engine_location",
           "wheel_base", "length", "width", "height", "curb_weight",
           "engine_type", "num_cylinders", "engine_size", "fuel_system",
           "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm",
           "city_mpg", "highway_mpg", "price"]

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data"
data = pd.read_csv(url, names=columns, na_values="?")

In [ ]:
data.shape

(205, 26)

In [ ]:
data.isna().sum()

,0
symboling,0
normalized_losses,41
make,0
fuel_type,0
aspiration,0
num_doors,2
body_style,0
drive_wheels,0
engine_location,0
wheel_base,0


In [ ]:
data = data.dropna(subset=["price"])

In [ ]:
for col in data.columns:
    if data[col].dtype != "object":
        data[col].fillna(data[col].astype(float).mean(), inplace=True)
    else:
        data[col].fillna(data[col].mode()[0], inplace=True)


/tmp/ipython-input-2001482774.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].astype(float).mean(), inplace=True)
/tmp/ipython-input-2001482774.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=

In [ ]:
# (i) num_doors, num_cylinders: words → numbers
word_to_num = {
    "two": 2, "three": 3, "four": 4, "five": 5,
    "six": 6, "eight": 8, "twelve": 12
}
data["num_doors"] = data["num_doors"].replace(word_to_num).astype(int)
data["num_cylinders"] = data["num_cylinders"].replace(word_to_num).astype(int)

# (ii) body_style, drive_wheels → dummy encoding
data = pd.get_dummies(data, columns=["body_style", "drive_wheels"], drop_first=True)

# (iii) make, aspiration, engine_location, fuel_type → label encoding
for col in ["make", "aspiration", "engine_location", "fuel_type"]:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])

# (iv) fuel_system to 1 if contains "pfi", else 0
data["fuel_system"] = data["fuel_system"].apply(lambda x: 1 if "pfi" in x else 0)

# (v) engine_type to 1 if contains "ohc", else 0
data["engine_type"] = data["engine_type"].apply(lambda x: 1 if "ohc" in x else 0)

data.shape

/tmp/ipython-input-2039684608.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["num_doors"] = data["num_doors"].replace(word_to_num).astype(int)
/tmp/ipython-input-2039684608.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["num_cylinders"] = data["num_cylinders"].replace(word_to_num).astype(int)


(201, 30)

In [ ]:
X = data.drop(columns=["price"])
y = data["price"].astype(float)

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

In [ ]:
r2 = r2_score(y_test, y_pred)

In [ ]:
r2

0.8732775682086301

In [ ]:
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)

In [ ]:
Xp_train, Xp_test, yp_train, yp_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

lr_pca = LinearRegression()
lr_pca.fit(Xp_train, yp_train)
yp_pred = lr_pca.predict(Xp_test)

In [ ]:
r2_pca = r2_score(yp_test, yp_pred)

In [ ]:
r2_pca

0.8611839960452383